### Setup

In [1]:
import numpy as np
import pandas as pd
import pickle
from JL_baseline import StreamingJL
from StreamingANN import StreamingANN
import psutil, os, time, json
from tqdm import tqdm
import gc

In [2]:
import numpy as np

def load_fvecs(fname):
    # Read raw data as int32 (because first value of each record is dimension)
    fv = np.fromfile(fname, dtype=np.int32)
    d = fv[0]  # first entry is the dimension
    fv = fv.reshape(-1, d + 1).astype(np.float32)
    return fv[:, 1:]  # drop the dimension prefix


In [3]:
dataset = load_fvecs('data/sift_base.fvecs')

In [37]:
import numpy as np

# Normalize first 5000 dataset points
mat = np.vstack([df[i].astype(np.float32) for i in range(5000)])
mat_norm = mat 

# Normalize query (first point)
q = df[5257].astype(np.float32)
q_norm = q

# Compute distances
dists = np.linalg.norm(mat_norm - q_norm, axis=1)

print("Min:", dists.min(), " Max:", dists.max(), " Mean:", dists.mean())

# this tells us that r should be about 0.5 - otherwise trivial :( 

Min: 0.39071167  Max: 1.3215674  Mean: 0.8274192


In [11]:
files = ["data/encodings.npy", "data/encodings_2.npy", "data/encodings_3.npy", "data/encodings_4.npy"]
arrays = [np.load(f) for f in files]
data = np.vstack(arrays)  
print(data.shape)

(80000, 384)


In [12]:
print(data.dtype)

float32


In [15]:
np.linalg.norm(data[1])

np.float32(1.0)

In [4]:
np.save("data/encodings_combined.npy", data)
print("Saved combined array to data/encodings_combined.npy")


Saved combined array to data/encodings_combined.npy


We precompute and store all the nearest neighbours for each point because that saves us a lot of time while running experiments over the entire dataset

In [30]:
import numpy as np

def suggest_r(data, sample_size=10000, percentile=5, seed=42):
    """
    Suggest a good inner radius `r` for StreamingANN / JL based on the dataset.

    Parameters
    ----------
    data : np.ndarray
        Dataset array of shape (N, d)
    sample_size : int
        Number of random pairs to sample for estimating distances
    percentile : float
        Percentile of the distance distribution to pick as `r`
        (e.g., 5 means the nearest 5% of points define "close")
    seed : int
        Random seed for reproducibility

    Returns
    -------
    r : float
        Suggested radius
    """

    rng = np.random.default_rng(seed)
    N = len(data)
    
    # If dataset is small, compute all pairwise distances
    if N*(N-1)//2 <= sample_size:
        # Full pairwise distances
        dists = np.linalg.norm(data[:, None, :] - data[None, :, :], axis=-1)
        dists = dists[np.triu_indices(N, k=1)]
    else:
        # Sample random pairs
        idx1 = rng.integers(0, N, size=sample_size)
        idx2 = rng.integers(0, N, size=sample_size)
        # Avoid identical points
        mask = idx1 != idx2
        idx1, idx2 = idx1[mask], idx2[mask]
        dists = np.linalg.norm(data[idx1] - data[idx2], axis=1)

    # Pick the desired percentile
    r = np.percentile(dists, percentile)
    print(f"Suggested r (inner radius) based on {percentile}th percentile: {r:.4f}")
    return r

suggest_r(df)

Suggested r (inner radius) based on 5th percentile: 0.5158


np.float32(0.51577383)

In [28]:
def load_fashion_mnist_csv(fname):
    """
    Load Fashion-MNIST CSV file.
    Assumes first column is label, remaining 784 columns are pixel values.
    Scales pixel values to [0,1] and applies L2 normalization.
    """
    df = pd.read_csv(fname)
    data = df.iloc[:, 1:].to_numpy(dtype=np.float32)  # drop label column

    # # Scale pixel values to [0,1]
    # data /= 255.0

    # Apply L2 normalization row-wise
    norms = np.linalg.norm(data, axis=1, keepdims=True)
    norms[norms == 0] = 1.0  # avoid division by zero
    data = data / norms

    return data


In [29]:
df = load_fashion_mnist_csv("data/fashion_mnist/fashion-mnist_train.csv")

In [3]:
# Load it later
with open("data/nn_dict.pkl", "rb") as f:
    loaded_nn_dict = pickle.load(f)

print("Loaded nn_dict has", len(loaded_nn_dict), "entries")

Loaded nn_dict has 80000 entries


### Plotting

### Evaluating the Basic Setup 
- Vary parameters r, epsilon, eta, n
- For each query q, check if the returned nearest neighbor is within r, if YES, we must check if our algorithm returns a valid approximate neighbour
- Want to check the basic scheme itself : how it performs (success rate) with increasing n, more aggressive sampling, and wider error margins 

In [ ]:
def load_completed_params(log_file):
    """Load completed (r, eps, eta, n, rep) tuples from a JSONL log file."""
    done = set()
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            for line in f:
                try:
                    rec = json.loads(line)
                    key = (rec["r"], rec["epsilon"], rec["eta"], rec["n"], rec["repeat"])
                    done.add(key)
                except json.JSONDecodeError:
                    continue
    return done


def evaluate_streaming_ann(
    data, nn_dict, r_list, eps_list, eta_list, n_list,
    repeats=3, n_queries=10000, seed=42,
    log_file="streaming_ann_log.jsonl", checkpoint_every=10
):
    """
    Run StreamingANN experiments with periodic checkpointing and resume support.

    Parameters
    ----------
    data : np.ndarray
        Dataset array (N,d)
    nn_dict : dict
        Maps each vector index in data to its nearest neighbor
    r_list, eps_list, eta_list, n_list : list
        Parameter ranges
    repeats : int
        Number of repeats per configuration
    n_queries : int
        Fixed number of queries per run
    log_file : str
        Path to JSONL log file for checkpointing
    checkpoint_every : int
        Write to disk after every X results
    """
    rng = np.random.default_rng(seed)
    proc = psutil.Process(os.getpid())
    results_buffer = []
    results = []

    # Load completed experiments (resume support)
    completed = load_completed_params(log_file)
    print(f"Found {len(completed)} completed runs in {log_file}, skipping them.")

    # Open log file in append mode
    f = open(log_file, "a")

    # total number of runs for progress bar
    total_runs = len(r_list) * len(eps_list) * len(eta_list) * len(n_list) * repeats

    with tqdm(total=total_runs, desc="StreamingANN experiments") as pbar:
        counter = 0
        for r in r_list:
            for eps in eps_list:
                for eta in eta_list:
                    for n in n_list:
                        if n > len(data):
                            continue
                        for rep in range(repeats):
                            key = (r, eps, eta, n, rep)
                            counter += 1

                            # Skip if already completed
                            if key in completed:
                                pbar.update(1)
                                continue

                            pbar.set_postfix({
                                "r": r, "eps": eps, "eta": eta, "n": n, "rep": rep
                            })

                            # --- dataset and query split ---
                            idx = rng.choice(len(data), size=n, replace=False)
                            dataset = data[idx]
                            remaining_idx = list(set(range(len(data))) - set(idx))
                            q_count = min(n_queries, len(remaining_idx))
                            q_idx = rng.choice(remaining_idx, size=q_count, replace=False)
                            queries = data[q_idx]

                            # --- memory snapshot before ANN build ---
                            mem_before = proc.memory_info().rss

                            # Build ANN
                            ann = StreamingANN(d=data.shape[1], n_estimate=n,
                                               eta=eta, epsilon=eps, r=r)
                            for p in dataset:
                                ann.insert(p)

                            # --- memory snapshot after ANN build ---
                            mem_after = proc.memory_info().rss
                            mem_ann_mb = (mem_after - mem_before) / 1024**2

                            # --- queries ---
                            successes = 0
                            valid_queries = 0
                            query_times = []

                            for q_global_idx, q in zip(q_idx, queries):
                                # lookup precomputed nearest neighbor index
                                nn_idx = nn_dict[q_global_idx]
                                nn_point = data[nn_idx]

                                # check if true neighbor is within radius r
                                if np.linalg.norm(nn_point - q) <= r:
                                    valid_queries += 1
                                    t0 = time.perf_counter()
                                    ann_res = ann.query(q)
                                    qtime = time.perf_counter() - t0
                                    query_times.append(qtime)
                                    if ann_res is not None:
                                        if np.linalg.norm(ann_res - q) <= (1+eps)*r:
                                            successes += 1

                            success_rate = (
                                successes / valid_queries if valid_queries > 0 else None
                            )

                            record = {
                                "r": r,
                                "epsilon": eps,
                                "eta": eta,
                                "n": n,
                                "repeat": rep,
                                "success_rate": success_rate,
                                "avg_query_time_ms": np.mean(query_times)*1000 if query_times else None,
                                "mem_usage_mb": mem_ann_mb,
                                "stored_points": len(ann.points),
                                "valid_queries": valid_queries
                            }

                            # After you're done using ann
                            del ann
                            gc.collect()
                            results_buffer.append(record)
                            results.append(record)

                            # Periodic checkpoint to file
                            if len(results_buffer) >= checkpoint_every:
                                for rec in results_buffer:
                                    f.write(json.dumps(rec) + "\n")
                                f.flush()
                                os.fsync(f.fileno())
                                results_buffer = []

                            pbar.update(1)

    # Write remaining results
    for rec in results_buffer:
        f.write(json.dumps(rec) + "\n")
    f.flush()
    f.close()

    print(f"Experiments finished. Results logged to {log_file}.")
    return results

In [ ]:
# --------------------------------------------------
# Define experiment parameter ranges
# Tune these ranges based on your dataset size and dimension
# --------------------------------------------------
r_list    = [1.0, 1.25, 1.5]       # candidate radii
eps_list  = [0.1, 0.2, 0.5, 1]         # epsilon values
eta_list  = [0.0, 0.1, 0.2, 0.5]     # sampling aggressiveness
n_list    = [1000, 10000, 30000, 50000]  # number of points from stream

# --------------------------------------------------
# Run experiments
# --------------------------------------------------
results = evaluate_streaming_ann(
    data,
    nn_dict = loaded_nn_dict,
    r_list=r_list,
    eps_list=eps_list,
    eta_list=eta_list,
    n_list=n_list,
    repeats=1,
    n_queries=10000,
    checkpoint_every=1,
    log_file="streaming_ann_log.jsonl"
)

# --------------------------------------------------
# 4. Save results to CSV
# --------------------------------------------------
df = pd.DataFrame(results)
df.to_csv("streaming_ann_results.csv", index=False)
print("Saved results to streaming_ann_results.csv")


Found 0 completed runs in streaming_ann_log.jsonl, skipping them.


StreamingANN experiments:   0%|          | 0/8 [00:00<?, ?it/s, r=1, eps=0.2, eta=0.2, n=1000, rep=0]

[Init] Inputs : eta=0.20, epsilon=0.20, r=1.00
[Init] LSH Parameters : w=2.785, p1=0.7141, p2=0.6591, rho=0.8080
[Init] Data Structure Parameters : k=17, L=371


StreamingANN experiments:  12%|█▎        | 1/8 [03:11<22:18, 191.18s/it, r=1, eps=0.2, eta=0.2, n=1e+4, rep=0]

[Init] Inputs : eta=0.20, epsilon=0.20, r=1.00
[Init] LSH Parameters : w=2.785, p1=0.7141, p2=0.6591, rho=0.8080
[Init] Data Structure Parameters : k=23, L=2389


...

[Init] Inputs : eta=0.20, epsilon=0.20, r=1.00
[Init] LSH Parameters : w=2.785, p1=0.7141, p2=0.6591, rho=0.8080
[Init] Data Structure Parameters : k=25, L=5806


StreamingANN experiments:  38%|███▊      | 3/8 [2:17:32<4:48:24, 3460.95s/it, r=1, eps=0.2, eta=0.2, n=5e+4, rep=0]

[Init] Inputs : eta=0.20, epsilon=0.20, r=1.00
[Init] LSH Parameters : w=2.785, p1=0.7141, p2=0.6591, rho=0.8080
[Init] Data Structure Parameters : k=26, L=8773


StreamingANN experiments:  38%|███▊      | 3/8 [38:07:00<63:31:41, 45740.22s/it, r=1, eps=0.2, eta=0.2, n=5e+4, rep=0]


KeyboardInterrupt: 

### Evaluating JL Scheme

In [4]:
def load_completed_jl_params(log_file):
    """Load completed (c, r, delta, n, rep) tuples from a JSONL log file."""
    done = set()
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            for line in f:
                try:
                    rec = json.loads(line)
                    key = (rec["c"], rec["r"], rec["delta"], rec["n"], rec["repeat"])
                    done.add(key)
                except json.JSONDecodeError:
                    continue
    return done


def evaluate_streaming_jl(
    data, nn_dict, c_list, r_list, delta_list, n_list,
    repeats=3, n_queries=10000, seed=42,
    log_file="streaming_jl_log.jsonl", checkpoint_every=10
):
    """
    Run StreamingJL experiments with periodic checkpointing and resume support.

    Parameters
    ----------
    data : np.ndarray
        Dataset array (N,d)
    nn_dict : dict
        Maps each vector index in data to its nearest neighbor
    c_list, r_list, delta_list, n_list : list
        Parameter ranges
    repeats : int
        Number of repeats per configuration
    n_queries : int
        Number of queries per run
    log_file : str
        Path to JSONL log file for checkpointing
    checkpoint_every : int
        Write to disk after every X results
    """
    rng = np.random.default_rng(seed)
    proc = psutil.Process(os.getpid())
    results_buffer = []

    completed = load_completed_jl_params(log_file)
    print(f"Found {len(completed)} completed runs in {log_file}, skipping them.")

    f = open(log_file, "a")
    total_runs = len(c_list) * len(r_list) * len(delta_list) * len(n_list) * repeats

    with tqdm(total=total_runs, desc="StreamingJL experiments") as pbar:
        for c in c_list:
            for r in r_list:
                for delta in delta_list:
                    for n in n_list:
                        if n > len(data):
                            continue
                        for rep in range(repeats):
                            key = (c, r, delta, n, rep)
                            if key in completed:
                                pbar.update(1)
                                continue

                            pbar.set_postfix({"c": c, "r": r, "delta": delta, "n": n, "rep": rep})

                            # --- dataset and query split ---
                            idx = rng.choice(len(data), size=n, replace=False)
                            dataset = data[idx]
                            remaining_idx = list(set(range(len(data))) - set(idx))
                            q_count = min(n_queries, len(remaining_idx))
                            q_idx = rng.choice(remaining_idx, size=q_count, replace=False)
                            queries = data[q_idx]

                            # --- memory before JL ---
                            mem_before = proc.memory_info().rss

                            # Build StreamingJL
                            jl_ann = StreamingJL(d=data.shape[1], n_max=n, c=c, r=r, delta=delta, random_state=seed)
                            for i, p in enumerate(dataset):
                                jl_ann.insert(p, i)

                            mem_after = proc.memory_info().rss
                            mem_jl_mb = (mem_after - mem_before) / 1024**2

                            # --- query evaluation ---
                            successes = 0
                            valid_queries = 0
                            query_times = []

                            for q_global_idx, q in zip(q_idx, queries):
                                nn_idx = nn_dict[q_global_idx]
                                nn_point = data[nn_idx]

                                if np.linalg.norm(nn_point - q) <= r:
                                    valid_queries += 1
                                    t0 = time.perf_counter()
                                    ans_id = jl_ann.query(q)
                                    qtime = time.perf_counter() - t0
                                    query_times.append(qtime)
                                    if ans_id is not None:
                                        dist = np.linalg.norm(dataset[ans_id] - q)
                                        if dist <= c * r:
                                            successes += 1

                            success_rate = (successes / valid_queries) if valid_queries > 0 else None

                            record = {
                                "c": c,
                                "r": r,
                                "delta": delta,
                                "n": n,
                                "repeat": rep,
                                "success_rate": success_rate,
                                "avg_query_time_ms": np.mean(query_times) * 1000 if query_times else None,
                                "mem_usage_mb": mem_jl_mb,
                                "stored_points": len(jl_ann.points),
                                "valid_queries": valid_queries
                            }

                            del jl_ann
                            gc.collect()
                            results_buffer.append(record)

                            if len(results_buffer) >= checkpoint_every:
                                for rec in results_buffer:
                                    f.write(json.dumps(rec) + "\n")
                                f.flush()
                                os.fsync(f.fileno())
                                results_buffer = []

                            pbar.update(1)

    # Write remaining results
    for rec in results_buffer:
        f.write(json.dumps(rec) + "\n")
    f.flush()
    f.close()

    print(f"JL experiments finished. Results logged to {log_file}.")


In [5]:
import numpy as np
import pandas as pd
import psutil
import os
import json
import gc
import time
from tqdm import tqdm

# Make sure your StreamingJL and JLTransformer classes are already defined
# Also make sure evaluate_streaming_jl and load_completed_jl_params are defined

# ------------------------------
# Experiment parameter ranges
# ------------------------------
c_list     = [1.2]                # JL scaling factor
r_list     = [1.0]                # candidate radii
delta_list = [0.1]                # JL failure probability
n_list     = [1000, 10000, 30000, 50000]  # number of points from stream

# ------------------------------
# Run experiments
# ------------------------------
jl_results_file = "streaming_jl_log.jsonl"

evaluate_streaming_jl(
    data,
    nn_dict=loaded_nn_dict,
    c_list=c_list,
    r_list=r_list,
    delta_list=delta_list,
    n_list=n_list,
    repeats=1,
    n_queries=10000,
    checkpoint_every=1,
    log_file=jl_results_file
)

# ------------------------------
# Load results from log file
# ------------------------------
jl_results = []
with open(jl_results_file, "r") as f:
    for line in f:
        try:
            jl_results.append(json.loads(line))
        except json.JSONDecodeError:
            continue

# ------------------------------
# Save results to CSV
# ------------------------------
df_jl = pd.DataFrame(jl_results)
df_jl.to_csv("streaming_jl_results.csv", index=False)
print("Saved results to streaming_jl_results.csv")


Found 0 completed runs in streaming_jl_log.jsonl, skipping them.


StreamingJL experiments:  25%|██▌       | 1/4 [3:18:53<9:56:39, 11933.06s/it, c=1.2, r=1, delta=0.1, n=1e+4, rep=0]


KeyboardInterrupt: 

### QPS vs Recall Benchmarking 

### Memory vs Recall Benchmarking